<a href="https://colab.research.google.com/github/dileep9968/pytorch/blob/main/03_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import torchmetrics, mlxtend
  print(f"torchmetrics version: {torchmetrics.__version__}")
  print(f"mlxtend version: {mlxtend.__version__}")
except:
  !pip install torchmetrics mlxtend
  import torchmetrics, mlxtend
  print(f"torchmetrics version: {torchmetrics.__version__}")
  print(f"mlxtend version: {mlxtend.__version__}")
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from timeit import default_timer as timer
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix
import requests
from pathlib import Path
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## 1. Getting a dataset

In [ ]:
train_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
    target_transform = None
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
)

In [ ]:
len(train_data),len(test_data)

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
image,label = train_data[0]
print(f"Images shape: {image.shape}->[channel, height, width]")
print(f"Image label: {class_names[label]}")

In [ ]:
train_data.targets

### Visualizing out data

In [ ]:
### Visualizeing our data
image , label = train_data[0]

print(f"Image shape:{image.shape}")
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
plt.imshow(image.squeeze(),cmap='gray')
plt.title(label)
plt.axis(False)

In [ ]:
# Plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4,4
for i in range(1, rows*cols+1):
  random_idx = torch.randint(0, len(train_data), size=[1]).item()
  img, label = train_data[random_idx]
  fig.add_subplot(rows, cols, i)
  plt.imshow(img.squeeze(), cmap='gray')
  plt.axis(False)
  plt.title(class_names[label])

### Prepared DataLoader
1. It is more computationlly efficeint as in your computing hardware
2. nn more chance to update its gradents per epoch



In [ ]:
train_dataloader = DataLoader(dataset = train_data,
                              batch_size=32,
                              shuffle=True)
test_dataloader = DataLoader(dataset = test_data,
                             batch_size=32,
                             shuffle = False)
train_dataloader, test_dataloader


In [ ]:
# Let's check out what we have created
print(f"Dataloader: {train_dataloader, test_dataloader}")
print(f"Lenght of train dataloader: {len(train_dataloader)} batch of 32")
print(f"Lenght of test dataloader: {len(test_dataloader)} batch of 32")

In [ ]:
train_features_batch, train_label_batch = next(iter(train_dataloader))
train_features_batch.shape, train_label_batch.shape

In [ ]:
# Show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_label_batch[random_idx]
plt.imshow(img.squeeze(), cmap='gray')
plt.title(class_names[label])
plt.axis(False)
print(f"image size : {img.shape}")
print(f"Label: {label}, label_size: {label.shape}")

In [ ]:

# Download helper function from pytroch repo
if Path('helper_functions.py').is_file():
  print('helper_functions.py already exists')
else:
  print('Downloading helper_functions.py')
  request = requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')
  with open('helper_functions.py','wb') as f:
    f.write(request.content)
from helper_functions import accuracy_fn


### Creating a function to time our experiments


In [ ]:

def print_train_time(start:float,
                     end:float,
                     device: torch.device=None):
  total_time = end-start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [ ]:
start_time = timer()

end_time = timer()
print_train_time(start=start_time,end=end_time,device='cpu')

### Creating a traning loopa and traning a model on batches of data
1. Loop through epochs
2. Loop through training batches perform traning steps, calculate training loss
3. loop thrugh testing batches step, calculate testing loss
4. print out metrics

## Convolutional Layer

In [ ]:
# Create a convolutional neural network
class FashionMNISTModelCNN(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels = hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels = hidden_units,
                  kernel_size=3,
                  padding=1,
                  stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                padding=1,
                stride=1
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                padding=1,
                stride=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*7*7,
                  out_features = output_shape)
    )

  def forward(self, x:torch.Tensor)->torch.Tensor:
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))


In [ ]:
torch.manual_seed(42)
model_cnn = FashionMNISTModelCNN(input_shape=1,
                                 hidden_units=10,
                                 output_shape=len(class_names)).to(device)
model_cnn

In [ ]:
torch.manual_seed(42)

def eval_model(model:torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device='cpu'):
  loss, acc = 0,0
  model.eval()
  with torch.inference_mode():
    for x, y in data_loader:
      x,y = x.to(device),y.to(device)
      y_pred = model(x)

      loss += loss_fn(y_pred,y)
      acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
    loss /= len(data_loader)
    acc /= len(data_loader)

  return {'Model_name ':model.__class__.__name__,
          'Model_loss ':loss.item(),
          "Model_acc ": acc}

### Functionzing training and testing loops

In [ ]:
def train_step(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device=device):
  train_loss, train_acc = 0,0
  model.train()
  for batch, (x,y) in enumerate(data_loader):
    x,y = x.to(device), y.to(device)

    y_pred = model(x)

    loss = loss_fn(y_pred,y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  # train loss and acc by length of train dataloader
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)

  print(f" Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")

In [ ]:
def test_step(model:torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device:torch.device=device):
  test_loss, test_acc = 0,0

  model.eval()
  with torch.inference_mode():
    for x, y in data_loader:
      x, y = x.to(device), y.to(device)

      test_pred = model(x)

      test_loss += loss_fn(test_pred, y)
      test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}%\n")

### Setup loss funciton/eval/optimizer

In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_cnn.parameters(),lr=0.01)


### Traning and testing

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Measure time
from timeit import default_timer as timer

train_time_start_modelcnn = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")
  train_step(model = model_cnn,
             data_loader = train_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             accuracy_fn = accuracy_fn,
             device = device)

  test_step(model = model_cnn,
            data_loader = test_dataloader,
            loss_fn = loss_fn,
            accuracy_fn = accuracy_fn,
            device = device)
  train_time_end_modelcnn = timer()
  total_train_time_model_cnn = print_train_time(start=train_time_start_modelcnn,
                                                 end=train_time_end_modelcnn,
                                                 device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
-------
 Train loss: 1.03780 | Train acc: 62.35%
Test loss: 0.62496 | Test acc: 76.93%

Train time on cpu: 65.991 seconds
Epoch: 1
-------
 Train loss: 0.54948 | Train acc: 80.01%


In [ ]:
model_cnn_result = eval_model(
    model=model_cnn,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    device=device
)

In [ ]:
model_cnn_result

In [ ]:
def make_predictions(model:torch.nn.Module,
                     data: list,
                     device: torch.device):
  pred_probs = []
  model.eval()
  with torch.inference_mode():
    for sample in data:
      sample = torch.unsqueeze(sample, dim=0)
      sample = sample.to(device)
      pred_logit = model(sample)
      pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)
      pred_probs.append(pred_prob.cpu())
  return torch.stack(pred_probs)

In [ ]:
import random
random.seed(42)
test_samples =[]
test_labels = []
for sample, label in random.sample(list(test_data), k=9):
  test_samples.append(sample)
  test_labels.append(label)



In [ ]:
test_samples[0].shape

In [ ]:
pred_probs = make_predictions(model = model_cnn,
                 data = test_samples,
                  device= device
                              )


In [ ]:
# convert prediciton probabilites to labels
pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
test_labels

In [ ]:
plt.figure(figsize=(9,9))
nrows =3
ncols =3
for i, sample in enumerate(test_samples):
  plt.subplot(nrows,ncols,i+1)
  plt.imshow(sample.squeeze(), cmap='gray')
  pred_label = class_names[pred_classes[i]]
  truth_label = class_names[test_labels[i]]
  titel_text = f"Pred: {pred_label} | Truth: {truth_label}"

  if pred_label == truth_label:
    plt.title(titel_text, fontsize=10, c='g')
  else:
    plt.title(titel_text, fontsize=10, c='r')
  plt.axis(False)


### Making confusion matrix

In [ ]:
import mlxtend
mlxtend.__version__

In [ ]:
# make predciton with trained model
y_preds = []
model_cnn.eval()
with torch.inference_mode():
  for x, y in tqdm(test_dataloader, desc='Making prediciton....'):
    x, y = x.to(device), y.to(device)
    y_logits = model_cnn(x)
    y_pred = torch.softmax(y_logits.squeeze(), dim=0).argmax(dim=1)
    y_preds.append(y_pred.cpu())

y_preds = torch.cat(y_preds)


In [ ]:
y_pred[:10]

In [ ]:


confmat = ConfusionMatrix(num_classes=len(class_names),task='multiclass')
confmat_tensor = confmat(y_preds, test_data.targets)

fig,ax = plot_confusion_matrix(conf_mat=confmat_tensor.numpy(),
                               class_names=class_names,
                               figsize=(10,7),
                               )

### Save and load model

In [ ]:
from pathlib import Path

MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = 'fashion_mnist_model.pth'
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

print(f"saving the model")
torch.save(obj=model_cnn.state_dict(),
           f=MODEL_SAVE_PATH)


In [ ]:
# create a new instance
torch.manual_seed(42)

load_model_cnn = FashionMNISTModelCNN(input_shape=1,
                                   hidden_units=10,
                                   output_shape=len(class_names)).to(device)
load_model_cnn.load_state_dict(torch.load(MODEL_SAVE_PATH))
load_model_cnn.to(device)


In [ ]:
laod_model_cnn_result = eval_model(
    model=load_model_cnn,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    device=device
)
laod_model_cnn_result

In [ ]:
model_cnn_result